In [11]:
# 20MIS0321 - AI Assignment 3

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
import numpy as np

In [13]:
# Here we are performing Data augumentation
train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))  #--> (0 to 255) convert to (0 to 1)
     

# Assigning data to variables.
train = train_gen.flow_from_directory('archive/train_data/train_data',
                                      target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=16)
test = test_gen.flow_from_directory('archive/test_data/test_data',
                                    target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=16)
     

Found 152 images belonging to 16 classes.
Found 158 images belonging to 16 classes.


In [14]:
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [4]:
# - - - M o d e l - - - 

model = Sequential()

#Input Conv Layer
model.add(Convolution2D(12,(3,3),activation='relu',input_shape=(120, 120, 3)))

# Normalizing using Batch Normalization
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#---------------------------------------------------------------------------------
model.add(Convolution2D(24,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#---------------------------------------------------------------------------------
model.add(Convolution2D(36,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#---------------------------------------------------------------------------------

model.add(Flatten()) #Flatterning into vector


# Hiddern layers -----------------------------------------------------------------
model.add(Dense(62,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.1))

model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))


# Output final layer -------------------------------------------------------------
model.add(Dense(16,activation='softmax'))


In [116]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train,batch_size=16,validation_data=test,epochs=50)

Epoch 1/50
10/10 [==============================] - 37s 4s/step - loss: 2.9552 - accuracy: 0.0658 - val_loss: 2.7916 - val_accuracy: 0.0382
Epoch 2/50
10/10 [==============================] - 34s 4s/step - loss: 2.5762 - accuracy: 0.1711 - val_loss: 2.7611 - val_accuracy: 0.0382
Epoch 3/50
10/10 [==============================] - 34s 4s/step - loss: 2.4164 - accuracy: 0.2566 - val_loss: 2.7602 - val_accuracy: 0.1338
Epoch 4/50
10/10 [==============================] - 34s 4s/step - loss: 2.3102 - accuracy: 0.2961 - val_loss: 2.7628 - val_accuracy: 0.1465
Epoch 5/50
10/10 [==============================] - 34s 4s/step - loss: 2.1652 - accuracy: 0.3684 - val_loss: 2.7810 - val_accuracy: 0.1720
Epoch 6/50
10/10 [==============================] - 34s 4s/step - loss: 2.0517 - accuracy: 0.4211 - val_loss: 2.8450 - val_accuracy: 0.0955
Epoch 7/50
10/10 [==============================] - 34s 4s/step - loss: 2.0083 - accuracy: 0.4342 - val_loss: 2.8991 - val_accuracy: 0.0955
Epoch 8/50
10/10 [==

In [117]:
model.save('Birds_Model_2.h5')
     

In [1]:
# As we can not the model is overfitting for the given dataset

In [2]:
# Now we try using transfer learning technique and lets see the difference

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

#Using VGG16 pre trained model and using imagenet weights
trained_model = VGG16(weights="imagenet", include_top=False, input_shape=(120, 120, 3))
trained_model.trainable = False


In [5]:
# here we are adding last few layers accourding to our dataset

flatten = layers.Flatten()
dense1 = layers.Dense(50, activation='relu')
dense2 = layers.Dense(20, activation='relu')
pred = layers.Dense(16, activation='softmax') #16 classes are there


model = models.Sequential([
    trained_model,
    flatten,
    dense1,
    dense2,
    pred
])

In [18]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.fit(train,batch_size=16,validation_data=test,epochs=25)

Epoch 1/25
10/10 [==============================] - 37s 4s/step - loss: 0.0730 - accuracy: 0.9934 - val_loss: 4.3102 - val_accuracy: 0.3608
Epoch 2/25
10/10 [==============================] - 37s 4s/step - loss: 0.0498 - accuracy: 0.9868 - val_loss: 4.5394 - val_accuracy: 0.3797
Epoch 3/25
10/10 [==============================] - 37s 4s/step - loss: 0.0424 - accuracy: 0.9934 - val_loss: 4.4172 - val_accuracy: 0.3608
Epoch 4/25
10/10 [==============================] - 37s 4s/step - loss: 0.0286 - accuracy: 1.0000 - val_loss: 4.3620 - val_accuracy: 0.4051
Epoch 5/25
10/10 [==============================] - 37s 4s/step - loss: 0.0184 - accuracy: 1.0000 - val_loss: 4.5013 - val_accuracy: 0.3671
Epoch 6/25
10/10 [==============================] - 38s 4s/step - loss: 0.0142 - accuracy: 1.0000 - val_loss: 4.5983 - val_accuracy: 0.3608
Epoch 7/25
10/10 [==============================] - 38s 4s/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 4.6374 - val_accuracy: 0.3671
Epoch 8/25
10/10 [==

In [ ]:
# Now the overfitting is reduced a bit

In [19]:
model.save('Birds_Model_by_transfer_learning.h5') # saving trasfered learned model 

In [ ]:
# Testing Model - with test images

In [38]:
img1 = image.load_img('archive/test_data/test_data/brhkyt/D72_0479.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ["blasti", "bonegl", "brhkyt", "cbrtsh", "cmnmyn", "gretit", "hilpig", "himbul", "himgri", "hsparo", "indvul", "jglowl"
 ,"lbicrw", "mgprob", "rebimg", "wcrsrt"]
print(output[pred])

1/1 [==============================] - 0s 35ms/step
2
brhkyt


In [32]:
img1 = image.load_img('archive/test_data/test_data/cmnmyn/_D32_12428.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ["blasti", "bonegl", "brhkyt", "cbrtsh", "cmnmyn", "gretit", "hilpig", "himbul", "himgri", "hsparo", "indvul", "jglowl"
 ,"lbicrw", "mgprob", "rebimg", "wcrsrt"]
print(output[pred])

1/1 [==============================] - 0s 35ms/step
4
cmnmyn


In [33]:
img1 = image.load_img('archive/test_data/test_data/wcrsrt/100_4466.JPG',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ["blasti", "bonegl", "brhkyt", "cbrtsh", "cmnmyn", "gretit", "hilpig", "himbul", "himgri", "hsparo", "indvul", "jglowl"
 ,"lbicrw", "mgprob", "rebimg", "wcrsrt"]
print(output[pred])

1/1 [==============================] - 0s 37ms/step
15
wcrsrt


In [ ]:
#20MIS0321 - E n d